# Intro to Long Short-Term Memory (LSTMs)

In class, we learned about recurrent neural networks.

![RNN](./img/recurrent_neural_network.jpg) 

Recurrent neural networks are great because they have some sort of memory of the past, meaning they can handle sequential data. Not only can the input be read in sequentially, but output can also be generated sequentially through, for example, an encoder-decoder model RNN. Moreover, by applying a a fixed function at each step, there is no limit to the length of an input sequence to an RNN.

However, RNNs struggle in long sequences when the information needed comes from a long time ago in the sequence (long-term dependency). This is because error is backpropagated, and over many time steps (as in a long sequence), gradients can either explode or vanish. This prevents the model from learning correctly, if at all. 

Long short-term memory (LSTM) units aim to solve the problem of exploding/vanishing gradients by choosing parts of each input to "remember" at each step. By controlling what information and how much of it to store, LSTMs can learn long-term dependencies.

----
## Long Short-Term Memory
----

An LSTM unit has multiple parts. The most important part of an LSTM network is the cell state, which is a pathway that runs through the entire sequence. LSTMs can selectively add or remove information from the cell state through multiple gates.

![cell_state](./img/lstm_cell.png) 
![operations_key](./img/lstm_operations.png) 

There are three main gates: the forget gate, the input gate, and the output gate.

### Forget Gate

The first thing that happens in an LSTM unit is the forget gate decides how much information to keep in the cell state. In the below diagram:
- $x_t$ represents the next token in the sequence
- $h_{t-1}$ represents the output from the last LSTM unit
- $\sigma$ is the sigmoid activation function.
- The output $f_t$ is a value between 0 and 1, with 0 representing "get rid of all information" and 1 representing "keep all information." 

![forget_gate](./img/lstm_forget.png) 

$h_{t-1}$ and $x_t$ are concatenated and multiplied by weight $W_f$. Then a bias $b_f$ is added. Finally, a sigmoid function is applied, resulting in an $f_t$ between 0 and 1.

Next, the unit decides how much new information we will store in the cell state. The layer that decides this is called the input gate.

### Input Gate

![input_gate](./img/lstm_input.png) 

The input gate has two parts:
- $i_t$ decides which values in the cell state will be updated
- $\tilde{C_t}$ is a vector "candidate values" to be added to the cell state

First, $i_t$ is calculated by multiplying $h_{t-1}$ concatenated with $x_t$ by weight $W_i$. A bias $b_i$ is then added, and the sigmoid is applied to the sum. We then calculate $\tilde{C_t}$ by concatenating $h_{t-1}$ with $x_t$, multiplying the sum by weight $W_C$, then adding a bias $b_C$. Finally, $i_t$ and $\tilde{C_t}$ are multiplied.

After $f_t$, $i_t$ and $\tilde{C_t}$ are calculated, the cell state is updated.

![update_cell_state](./img/lstm_update_cell_state.png) 

### Output Gate

The last part of the LSTM unit is the output gate, which decides what the LSTM unit outputs.

![output_gate](./img/lstm_output.png) 

According to the formulas above, we first calculate $o_t$. A concatenated $h_{t-1}$ and $x_t$ is multiplied by $W_o$, then bias $b_o$ is added. Finally, the sigmoid is applied to the sum.

The output $h_t$ is then the product of $o_t$ and $\tanh(C_t)$.

### Summary

LSTM units work because they maintain a cell state that controls the gradient flow. The gates open and close, learning weights such that errors are appropriately scaled, trapped, and released. This facilitates a constant error flow that prevents gradients from exploding or vanishing.

## Implementation

Below we outline a possible implemenation of the LSTM class.

In [ ]:
import torch
from torch.nn import Parameter

class LSTM(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)

        # Initialize forget gate weights
        self.wf = Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bf = Parameter(torch.tensor(0.0), requires_grad=True)

        # Initialize input gate weights
        self.wi = Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bi = Parameter(torch.tensor(0.0), requires_grad=True)

        # Initialize candidate gate weights
        self.wc = Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bc = Parameter(torch.tensor(0.0), requires_grad=True)

        # Initialize output gate weights
        self.wo = Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bo = Parameter(torch.tensor(0.0), requires_grad=True)

    def lstm_unit(self, input, cell_state, short_memory):
        """Does the calculations for the LSTM unit."""
        
        # Calculating forget and input gate values
        ft = torch.sigmoid((self.wf * input) + (self.wf * short_memory) + self.bf)
        it = torch.sigmoid((self.wi * input) + (self.wi * short_memory) + self.bi)
        candidates = torch.tanh((self.wc * input) + (self.wc * short_memory) + self.bc)

        # Update the cell state
        updated_cell_state = (ft * cell_state) + (it * candidates)

        # Output gate
        ot = torch.sigmoid((self.wo * input) + (self.wo * short_memory) + self.bo)
        ht = torch.tanh(cell_state) * ot

        return updated_cell_state, ht
    
    def forward(self, input):
        """A forward pass. Here were assume input is an array/list.
           Depending on the needs, the inputs may need to be
           handled differently."""
        
        cell_state = 0
        short_memory = 0

        for item in input:
            cell_state, short_memory = self.lstm_unit(item, cell_state, short_memory)
            
        return short_memory

In the constructor, we initialize all the weights and biases. In this particular implementation, we randomly set the weights based on a normal distribution around mean 0 and standard deviation 1. The biases are initialized to 0.

Next, we define a helper function `lstm_unit` to perform the calculations that occur in the LSTM unit. Values are calculated based on the formulas above and the function returns the updated cell state and short term memory.

Finally, in the `forward` method, we iterate through the input and perform the LSTM calculations, returning the updated short term memory.

## Music Generation

We explore music generation using techniques similar to the charRNN model. The structure of our code is as follows:

- **`music_preprocessing.py`**: Converts `.mid` and `.krn` files into data trainable by LSTM.
- **`get_data.py`**: Transforms the training data into `torch.tensors`.
- **`buildLSTM.py`**: Defines and trains the neural network.
- **`generate_music.py`**: Generates music using the trained model.

### Data Pre-processing Challenge

One significant challenge in music generation is data pre-processing. Music is not in a readily usable form, requiring extensive pre-processing to convert it into trainable data.

**Step One**: Our dataset is in `.erk` format, and we use the `music21` library to convert music notation into numerical data. This process is handled in `music_preprocessing.py`. Here is a brief overview of its functionality:

- **Transposition**: Since our training dataset contains music in various keys, we first transpose the music to either C major or A minor.
- **Pitch Conversion**: After transposition, we convert the music to their respective pitches in string format. For more details, see the function `music_encoder`. Running this file generates a new folder `__data_preprocessing__` containing digital music converted to pitches.

After this step, our preprocessed music would look like this:

`62 _ _ _ _ _ _ _ r _ _ _ 64 _ 67 _ 65 _ _ _ 64 _ _ _`

where the numbers are pitch values, `_` denotes the temporal duration of the note/rest, and `r` denotes rest. To see its actual power, please run the line below: 

In [ ]:
from music_preprocessing import preprocess_midi_files
preprocess_midi_files()

**Step Two**: Transform the string of pitches into a dataset for training, handled in `get_data.py`.

- **String to Tokens**: We tokenize the string and store a dictionary of tokens. The string is then tokenized and one-hot encoded. Running this file generates a new folder `music_to_integer_dictionary` containing a dictionary of tokens for pitches.

**Step Three**: Implement LSTM for music generation, with a simple structure. Note that the input size and output size are equal to the vocabulary size in our training set. Below is our structure:

- **Neural Network Structure**: We pass the data through a one-layer LSTM model followed by a linear layer of size `(hidden_size * SEQUENCE_LENGTH, output_size)` and then apply the activation function. Some research articles show that stacked LSTMs would produce the best effect for music generation, but we consider a simple layer is sufficient for our purposes here and easier for training. You can read more [here](https://arxiv.org/abs/2203.12105)

In [ ]:
from torch import nn  
SEQUENCE_LENGTH = 64
class LSTMModel(nn.Module): 
    def __init__(self, input_size, output_size, hidden_size, 
                 number_of_layers, 
                 dropout_prob):  
        super().__init__()
        self.lstms = nn.LSTM(input_size, hidden_size, dropout = dropout_prob, batch_first = True, num_layers = number_of_layers)  
        self.dense = nn.Linear(hidden_size * SEQUENCE_LENGTH, output_size)
        self.number_of_layers = number_of_layers 
        self.hidden_size = hidden_size 

The structure of our neural network is extremely simple. We first pass the data into lstms and then a linear layer of size `(hidden_size * SEQUENCE_LENGTH, output_size)` and then perform the activation function. 

**Step Four**: After completing the training, we can then the evaluation step and generate music based on our seed. To give a simple example, the seed

`seed = "65 _ _ _ 65 _ 69"`

will provide music like the following: 
![music_sample](./img/music_screenshot.jpg) 

You can change the seed of your choice in the file `generate_music.py` and execute the file to generate your own music!

In the music generation process, we have implemented a temperature sampling which allows our ouput to be more random and thus generates creative music. This is basically redefining a new softmax function: 
$$
\text{softmax}(x_i) = \frac{e^{x_i / T}}{\sum^{n}_{j = 1}e^{x_j / T}}
$$
where $T$ denotes the hyperparameter for temperature value. As $T \rightarrow \infty$, we see that the process becomes more random; as $T \rightarrow 0$, we see the process becomes more deterministic and thus repetitive. To avoid repetitive generation, we are interested in a sufficiently high value of temperature. You can read more about it [here](https://towardsdatascience.com/how-to-sample-from-language-models-682bceb97277).


The implementation is shown below:

In [ ]:
import numpy as np 
import torch.nn.functional as F 

def sample_with_temperature(self, probabilites, temperature):
    '''
    Samples an index from a probability array using temperature sampling.
        
    Parameters:
        probabilities (torch.Tensor): Tensor of probabilities for each possible output.
        temperature (float): Controls the randomness of predictions by scaling the logits before applying softmax.
        
    Returns:
        index (int): The selected index based on the sampling.
        '''
    probabilites = probabilites.squeeze() 
    predictions = torch.log(probabilites) / temperature
    probabilites = F.softmax(predictions, dim = -1)
    choices = range(len(probabilites))
    probabilites = probabilites.detach().numpy()
    index = np.random.choice(choices, p = probabilites)
    return index

### Future Improvements and Considerations

The project is by no means complete, and many further improvements are possible. Here are some suggestions:

- Polyphonic Music

  - **Current Limitation**: Our music generation model currently only processes monophonic music.
  - **Future Exploration**: An interesting question would be how to incorporate polyphonic music into our model such as multi-layer voiced music, multiple instruments, and complex chords. Can we add more instruments to create chamber music? 
  - **Complexity Goal**: Can we generate music with the complexity as shown below?
  ![beethoven](./img/beethoven.jpg)

- Hyperparameter Tuning

  - **Current State**: We have provided some hyperparameters in the file `buildLSTM.py`.
  - **Future Work**: Try to find the best hyperparameters to generate beautiful music.
  - **Performance Improvement**: Furthermore, explore ways to improve the training speed. 

- Sampling Technique

  - **Current Issue**: A drawback in `sample_with_temperature` is that it can potentially disrupt the overall structure of the musical flow.
  - **Future Enhancement**: Is there a better sampling technique available that could make our music more coherent?

- Score Representation

  - **Current Limitation**: The current implementation for converting music is still a bit hard to read.
  - **Future Improvement**: Explore ways to make the score more visually appealing, such as dividing it into treble clef and bass clef.